In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize the images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Enhanced data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(x_train)

# Build the model
inputs = tf.keras.Input(shape=(32, 32, 3))
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=inputs)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)  # Dense layer with 512 units
x = Dropout(0.2)(x)  # Dropout layer with rate 0.2
outputs = Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.00027679),  # Set learning rate to 0.00027679
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Define callbacks
checkpoint = ModelCheckpoint('efficientnetb0_cifar10_best.h5.keras', monitor='val_accuracy', save_best_only=True, verbose=1, save_weights_only=False)


# Train the model
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=128),
    epochs=50,  # You can adjust this
    validation_data=(x_test, y_test),
    callbacks=[checkpoint]
)

# Save the trained model
best_model_path = 'best_model_after_training.h5.keras'
model.save(best_model_path)

# Load the best model
best_model = tf.keras.models.load_model(best_model_path)

# Evaluate the best model
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

# Save the model for distillation
distillation_model_path = 'student_model.h5.keras'
best_model.save(distillation_model_path)


In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import HeNormal
import keras_tuner as kt
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize the images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Enhanced data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(x_train)

# Define the hypermodel
class CIFAR10HyperModel(HyperModel):
    def build(self, hp):
        # Input layer
        inputs = tf.keras.Input(shape=(32, 32, 3))
        
        # Base model
        base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=inputs)
        x = base_model.output
        
        # Additional layers
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(hp.Int('units', min_value=384, max_value=384, step=128), activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1))(x)
        outputs = layers.Dense(10, activation='softmax')(x)
        
        model = models.Model(inputs=inputs, outputs=outputs)
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        return model

# Define the hyperparameter tuner
tuner = kt.BayesianOptimization(
    CIFAR10HyperModel(),
    objective='val_accuracy',
    num_initial_points=3,
    max_trials=3,
    directory='kt_dir',
    project_name='student_model_tuning',
    overwrite=True
)

# Define callbacks
checkpoint = ModelCheckpoint('efficientnetb0_cifar10_best.h5.keras', monitor='val_accuracy', save_best_only=True, verbose=1,save_weights_only=False)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)


# Perform the hyperparameter search
tuner.search(
    datagen.flow(x_train, y_train, batch_size=128),
    epochs=20,  # 20 epochs for each trial
    validation_data=(x_test, y_test)
)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Train the model with the best hyperparameters
history = best_model.fit(
    datagen.flow(x_train, y_train, batch_size=128),
    epochs=50,  # You can adjust this
    validation_data=(x_test, y_test),
    callbacks=[checkpoint, early_stopping]
)

best_model_path = 'best_model_after_tuning.h5.keras'
best_model.save(best_model_path)

# Load the best model
best_model = tf.keras.models.load_model(best_model_path)

# Evaluate the best model
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

# Save the model for distillation
distillation_model_path = 'student_model.h5.keras'
best_model.save(distillation_model_path)

# Additional training (if needed)
best_model = tf.keras.models.load_model(best_model_path)